In [1]:
import geopandas as gpd
from pystac_client import Client
from odc.stac import load

from dask.distributed import Client as DaskClient

from pathlib import Path

from numpy import nanpercentile

In [2]:
areas = gpd.read_file('areas.geojson')

areas.explore()

In [4]:
# Strip newlines
areas["Capital"] = areas["Capital"].str.strip()
areas["Country"] = areas["Country"].str.strip()
areas["Name"] = areas["Name"].str.strip()

# Select Country is Australia
one = areas[areas["Country"] == "Fiji_1"]
one

,Name,Country,Capital,Hometown,geometry
12,CROP DG's,Fiji_1,Suva,None,"POLYGON ((178.23214 -17.91509, 178.23214 -18.3..."


In [5]:
client = Client.open("https://earth-search.aws.element84.com/v1")
collection = "sentinel-2-c1-l2a"
datetime = "2024"

def mosaic_region(region, overwrite=True):
    output = Path(f"{region.Country.values[0]}.png")
    if output.exists() and not overwrite:
        print(f"Output file {output} already exists, skipping")
        return None

    dask_client = DaskClient(n_workers=2, threads_per_worker=16)
    print("Searching for items in region")
    items = client.search(
        collections=[collection],
        intersects=region.geometry.values[0],
        datetime=datetime,
        query={"eo:cloud_cover": {"lt": 80}},
    ).item_collection()
    print(f"Found {len(items)} items")

    data = load(
        items,
        geopolygon=region.geometry.values[0],
        measurements=["red", "green", "blue", "scl"],
        chunks={"x": 2048, "y": 2048},
        groupby="solar_day"
    )

    print(f"Loaded data with dimensions x: {data.x.size}, y: {data.y.size}, time: {data.time.size}")

    mask_flags = [0, 3, 8, 9]
    mask = data.scl.isin(mask_flags)

    data = data.where(~mask).drop_vars("scl")

    print("Computing median")
    median = data.median("time").compute()

    print("Working out percentile and making pretty picture")
    percentile_stretch = (1, 99)
    rgb_array = median.to_array().values
    vmin, vmax = nanpercentile(rgb_array, percentile_stretch)
    visualisation = median.odc.to_rgba(vmin=vmin, vmax=vmax).compute()

    dask_client.close()

    visualisation.odc.write_cog(output, overwrite=True)
    print(f"Saved visualisation to {output}")

    return visualisation


visulisation = mosaic_region(one)


Searching for items in region
